In [1]:
import os
from data import *

In [8]:
# check if Bertified data exists
if not os.path.isfile('../data/Bertified/entities.npy'):
    reverb_lines = read_reverb('../data/Reverb1.1/reverb_wikipedia_tuples-1.1.txt')
    questions = pd.read_excel('../data/ReverbSQA/Final_Sheet_990824.xlsx', sheet_name=1, engine='openpyxl')
    index = get_tuple_frequency(reverb_lines, questions)
    index[index['Frequency']<10].to_excel('../data/ProcessedQuestions/normalized_questions.xlsx')
    combine_with_reverb(questions_path='../data/ProcessedQuestions/normalized_questions.xlsx', 
                    reverb_path='../data/Reverb1.1/reverb_wikipedia_tuples-1.1.txt')
    create_bertified_dataset()


In [11]:
# table 1 output 
pd.read_excel('../data/Intermediate/train.xlsx').sample(6)[['triple', 'Question']]

,triple,Question
6196,"('Oracle Corporation', 'completed the acquisit...",What Oracle Corporation completed
7399,"('Lee', 'was elected mayor of', 'Chapel Hill')",Who was elected mayor of Chapel Hill
6300,"('The WEC', 'was first organized in', '1990')",What was first organized in 1990
4868,"('The Wall Street Journal', 'reported in', 'Se...",when did The Wall Street Journal report
9739,"('Clarendon Hills', 'is an affluent village in...",What village is in DuPage County
5658,"('Second-parent adoptions', 'were designed for...",for whom were Second-parent adoptions designed


In [17]:
# table 2 output
df = pd.read_csv(r'../data/Reverb1.1/reverb_wikipedia_tuples-1.1.txt', sep='\t', header=None)
reverb_columns_name = ['ExID', 'arg1', 'rel', 'arg2', 'narg1', 'nrel', 'narg2', 'csents', 'conf', 'urls']
df.columns = reverb_columns_name
df = df.dropna()
df = df.drop_duplicates()

print(f'#Triples\t:\t{len(df)}')
print(f'#Relations\t:\t{len(df["rel"].unique())}')
print(f'#Entity 1\t:\t{len(df["arg1"].unique())}')
print(f'#Entity 2\t:\t{len(df["arg2"].unique())}')
print(f'Total Unique Entities\t:\t{len(set(df["arg1"].unique().tolist()+df["arg2"].unique().tolist()))}')
vocab = df["arg1"].unique().tolist()+df["arg2"].unique().tolist()+df["rel"].unique().tolist()
vocab = list(map(lambda x:x.split(), vocab))
vocab = [item for sublist in vocab for item in sublist]
print(f'Vocabulary Size\t:\t{len(set(vocab))}')

#Triples	:	407236
#Relations	:	101977
#Entity 1	:	182329
#Entity 2	:	156166
Total Unique Entities	:	304853
Vocabulary Size	:	148953


In [31]:
# table 3 output
train_df = pd.read_excel('../data/Intermediate/train.xlsx'); valid_df = pd.read_excel('../data/Intermediate/valid.xlsx'); test_df = pd.read_excel('../data/Intermediate/test.xlsx')


def get_unique_ent_rel(dataframe):
    arg1 = [eval(item)[0] for item in dataframe['triple'].to_list()]
    arg2 = [eval(item)[2] for item in dataframe['triple'].to_list()]
    rel = [eval(item)[1] for item in dataframe['triple'].to_list()]
    print(f'Number of Questions\t:\t{len(dataframe)}')
    print(f'Entity 1\t:\t{len(set(arg1))}')
    print(f'Entity 2\t:\t{len(set(arg2))}')
    print(f'Relations\t:\t{len(set(rel))}')
    print(f'Total Unique Entities\t:\t{len(set(arg1+arg2))}')
    tokenizer = lambda string:string.strip().lower().split()
    tokenized_questions = dataframe['Question'].astype(str).apply(tokenizer).to_list()
    flatten_tokenized_questions = [item for sublist in tokenized_questions for item in sublist]
    print(f'Unique Words\t:\t{len(set(flatten_tokenized_questions))}')

print("*** Training ***")
get_unique_ent_rel(train_df)
print("\n*** Validation ***")
get_unique_ent_rel(valid_df)
print("\n*** Test ***")
get_unique_ent_rel(test_df)


*** Training ***
Number of Questions	:	9921
Entity 1	:	6549
Entity 2	:	5971
Relations	:	4568
Total Unique Entities	:	12155
Unique Words	:	10123

*** Validation ***
Number of Questions	:	1751
Entity 1	:	1624
Entity 2	:	1566
Relations	:	1297
Total Unique Entities	:	3147
Unique Words	:	2958

*** Test ***
Number of Questions	:	5003
Entity 1	:	4064
Entity 2	:	3827
Relations	:	2939
Total Unique Entities	:	7702
Unique Words	:	6314


In [14]:
# table 5 output {GRU or LSTM}
!python ./BuboQA/entities/train.py  --entity_detection_mode GRU \
                                    --fix_embed --data_dir ../data/SimpleQuestionNotationEntity \
                                    --batch_size 256 \
                                    --vector_cache ../data/Cache/sq_glove300d.pt \
                                    --no_cuda


Embedding match number 8546 out of 13972
Namespace(batch_size=256, clip_gradient=0.6, cuda=False, data_dir='../data/SimpleQuestionNotationEntity', dataset='EntityDetection', dev_every=2000, entity_detection_mode='GRU', epochs=30, gpu=-1, hidden_size=300, hits=100, input_size=300, label=4, log_every=1000, lr=0.0001, num_layer=2, patience=10, results_path='query_text', rnn_dropout=0.3, rnn_fc_dropout=0.3, save_path='saved_checkpoints', seed=3435, specify_prefix='id1', train_embed=False, trained_model='', vector_cache='../data/Cache/sq_glove300d.pt', weight_decay=0, words_dim=300, words_num=13972)
VOCAB num 13972
Train instance 9921
Dev instance 1751
Test instance 5003
Entity Type 4
EntityDetection(
  (embed): Embedding(13972, 300)
  (gru): GRU(300, 300, num_layers=2, dropout=0.3, bidirectional=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (relu): ReLU()
  (hidden2tag): Sequential(
    (0): Linear(in_features=600, out_features=600, bias=True)
    (1): BatchNorm1d(600, eps=1e-05, mome

In [7]:
# table 7 & 8 output


%cd ./src
# model types : [MultiDepthNodeEdgeDetector, BertLSTMCRF, BertCNN, NodeEdgeDetector]
# cross validation: [True, False]
!python train.py NodeEdgeDetector True




[Errno 2] No such file or directory: './src'
/mnt/d/git/CodeOceanBERTQA/code/src
INFO:root:

############# Fold Number 1 #############


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/pytorch_model.bin HTTP/1.1" 302 0
DEBUG:filelock:Attempting to acquire lock 139709743933136 on /home/meti/.cache/huggingface/hub/models--bert-base-uncased/blobs/097417381d6c7230bd9e3557456d726de6e83245ec8b24f529f60198a67b203a.lock
DEBUG:filelock:Lock 139709743933136 acquired on /home/meti/.cache/huggingface/hub/models--bert-base-uncased/blobs/097417381d6c7230bd9e3557456d726de6e83245ec8b24f529f60198a67b203a.lock
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): cd

In [8]:
from transformers import BertModel
BertModel.from_pretrained("bert-base-uncased")

KeyboardInterrupt: 

In [ ]:


%cd /content/BERT-QA/src
# model types : [MultiDepthNodeEdgeDetector, BertLSTMCRF, BertCNN, NodeEdgeDetector]
# cross validation: [True, False]
!python train.py NodeEdgeDetector True

